In [1]:
import numpy as np
import pandas as pd
import math
import time
import re
import os
from scipy.io import wavfile
from skimage import util
from scipy import signal
from scipy import stats

#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

#from sklearn.cluster import KMeans
#from sklearn.metrics.cluster import silhouette_score

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

In [2]:
def create_slice_from_wav(file_path, slice_len, step_size):
    """Creates small slices from wav file. Slice_len (use sampling frequency to convert to ms). 
    Step_size is amount of overlap between each slice."""
    
    #get animal name
    
    #read in wav file
    samp_freq, sig_data = wavfile.read(file_path)
    sig_data = sig_data[0:150000000]
    print('Sampling frequency: ' + str(samp_freq))
    
    #determine number of samples and length
    n_samples = sig_data.shape[0]
    print('Number of samples: ' + str(n_samples))
    sig_len = n_samples/samp_freq
    print('Length: ' + str(sig_len) + ' sec')
    
    #create slices 
    M = slice_len
    steps = int(M*step_size)
    slices = util.view_as_windows(sig_data, window_shape=(M,), step=steps)
    print(f'Audio shape: {sig_data.shape}, Sliced audio shape: {slices.shape}')
    
    return samp_freq, sig_data, slices, steps

In [3]:
def plot_spec(Sx, times, steps, time_stamp):
    """Plots a spectrogram from a slice"""
    
    f, ax = plt.subplots()
    plt.pcolormesh((times*1000) + (time_stamp), freqs_spec / 1000, 10 * np.log10(Sx), cmap = 'cubehelix')
    ax.ticklabel_format(useOffset=False)
    plt.ylabel('Frequency [kHz]')
    plt.xlabel('Time [msec]')
    plt.show()
    
    return plt

In [4]:
def multi_plot(image_df, time_stamp_list, x, y):
    """Plots spectrograms from a list of time_stamps"""
    for time_stamp in time_stamp_list[x:y]:
        plt.figure(figsize = (2,5))
        plt.pcolormesh((times*1000) + (time_stamp), freqs_spec / 1000, 10 * np.log10(image_df[time_stamp]), cmap = 'cubehelix')
        plt.show()

In [5]:
def find_features(data):
    """Finds spectral flatness and power sum for each time stamp in a df."""
    
    start = time.time()
    
    feature_df = pd.DataFrame(index = data.index, columns = ['time_stamp', 'spec_flat', 'power_sum'])
    
    for time_stamp in data.index:
        #spectral flattness
        feature_df.loc[time_stamp]['spec_flat'] = (stats.gmean(data.loc[time_stamp])) / (data.loc[time_stamp].mean())
        #power sum
        feature_df.loc[time_stamp]['power_sum'] = data.loc[time_stamp].sum()
        #time stamp
        feature_df.loc[time_stamp]['time_stamp'] = time_stamp
        
    end = time.time()
    print(end - start)

    return feature_df

In [6]:
wav_dir_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x'

In [7]:
path_names = []
files = os.listdir(wav_dir_path)
for file in files: 
        path_names.append(wav_dir_path + "/" + file)

path_names

['C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/533.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/534.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/535.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/542.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/543.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/554.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/555.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/559.wav',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/18.12.07_CPA_pair_3x/noise.wav']

Create slices from wav file

In [ ]:
spec_slices = {}
spec_slices_ravel = {}
spec_slices_df_dic = {}

spec_window = 128
NFFT = 512
    
for path in path_names:
    
    name = re.search("\d\d\d", path).group(0)
    
    print(str('Begin processing animal # ' + name))
    
    #create slices
    start = time.time()
    samp_freq, sig_data, slices, steps = create_slice_from_wav(path, 6250, 0.9)
    end = time.time()
    print(str('Slices created in ' + str(end - start) + '  seconds'))
    
    #create spectrogram from each slice
    start = time.time()
    i = 0
    samp_freq_kHz = samp_freq/1000
    spec_slices_int = {}
    spec_slices_ravel_int = {}
    
    for i in range(slices.shape[0]): 
        if i % 1000 == 0:
            print(i)
        #spectrogram
        freqs_spec, times, Sx = signal.spectrogram(slices[i,:], fs=samp_freq, nperseg = spec_window, nfft = NFFT)
    
        time_stamp = i*steps / samp_freq_kHz
    
        #store as intermediate dic
        spec_slices_int[time_stamp] = Sx
        spec_slices_ravel_int[time_stamp] = spec_slices_int[time_stamp].ravel().T
    
    end = time.time()
    print(str('Spectrograms created in ' + str(end - start) + '  seconds'))
    
    #store as dic of dic (animal number is highest dic key)
    start = time.time()
    spec_slices[name] = spec_slices_int
    spec_slices_ravel[name] = spec_slices_ravel_int
    
    spec_slices_df_int = pd.DataFrame(spec_slices_ravel_int).T
    spec_slices_df_dic[name] = spec_slices_df_int
    
    end = time.time()
    print(str('Data frame created in ' + str(end - start) + '  seconds'))

Begin processing animal # 533
Sampling frequency: 250000
Number of samples: 150000000
Length: 600.0 sec
Audio shape: (150000000,), Sliced audio shape: (26666, 6250)
Slices created in 0.25332117080688477  seconds
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
Spectrograms created in 294.86925506591797  seconds
Data frame created in 13.448246002197266  seconds
Begin processing animal # 534
Sampling frequency: 250000
Number of samples: 150000000
Length: 600.0 sec
Audio shape: (150000000,), Sliced audio shape: (26666, 6250)
Slices created in 9.161814212799072  seconds
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
Spectrograms created in 1530.364191532135  seconds
Data frame created in 16.332616806030273  seconds
Begin processing animal # 535
Sampling frequency: 250000
Number of samples: 1

In [10]:
spec_slices_df_dic[533]

KeyError: 533

Threshold data

In [ ]:
start = time.time()

spec_slices_thresh_df = pd.DataFrame(index = spec_slices_df.index)
for time_stamp in spec_slices_df.index:
    threshold = np.percentile(spec_slices_df.loc[time_stamp], 70)
    spec_slices_thresh_df.loc[time_stamp][spec_slices_df.loc[time_stamp] < threshold] = 0

end = time.time()
print(end - start)
spec_slices_thresh_df.head()    

Find spectral flatness of each spectrogram

In [ ]:
features_df = find_features(spec_slices_df)
print(features_df.shape)
features_df.head()

In [ ]:
start = time.time()

spec_flat_df = pd.DataFrame(index = spec_slices_df.index, columns = ['time_stamp', 'spec_flat'])
for time_stamp in spec_slices_df.index:
    x = spec_slices_df.loc[time_stamp]
    spec_flat_df.loc[time_stamp]['spec_flat'] = (stats.gmean(x)) / (x.mean())
    spec_flat_df.loc[time_stamp]['time_stamp'] = time_stamp
    
print(spec_flat_df.shape)
end = time.time()
print(end - start)

spec_flat_df.head(10)

In [ ]:
multi_plot(spec_slices, spec_flat_df.index.values)

Threshold on spectral flatness

In [ ]:
#threshold_flat = np.percentile(spec_flat_df['spec_flat'].values, 25)
#print(threshold_flat)
spec_flat_df_thresh = spec_flat_df[spec_flat_df['spec_flat'] < .1]
print(spec_flat_df_thresh.shape)
spec_flat_df_thresh.head()

Create new df of thresholded slices

In [ ]:
#time_stamp_list_power = power_sum_df_thresh.index.values
time_stamp_list_flatness = spec_flat_df_thresh.index.values

In [ ]:
spec_slices_df_flat_thresh = spec_slices_df.loc[[time_stamp_list_flatness][0]]
print(spec_slices_df_flat_thresh.shape)
spec_slices_df_flat_thresh.head()

In [ ]:
multi_plot(spec_slices, time_stamp_list_flatness)

Find power sum of thresholded slices

In [ ]:
start = time.time()

power_sum_df = pd.DataFrame(index = spec_slices_df_flat_thresh.index, columns = ['time_stamp', 'power_sum'])
for time_stamp in spec_slices_df_flat_thresh.index:
    power_sum_df.loc[time_stamp]['power_sum'] = spec_slices_df_flat_thresh.loc[time_stamp].sum()
    power_sum_df.loc[time_stamp]['time_stamp'] = time_stamp
    
print(power_sum_df.shape)
end = time.time()
print(end - start)

power_sum_df.head()

In [ ]:
power_sum_df.head()

In [ ]:
plt.hist(power_sum_df['power_sum'].values)
plt.show()

In [ ]:
power_sum_df.head()

In [ ]:
power_sum_df.max()

Threshold on power sum

In [ ]:
#threshold_power = np.percentile(power_sum_df['power_sum'].values, 20)
#print(threshold_power)
power_sum_df_thresh = power_sum_df[power_sum_df['power_sum'] > 200000]
print(power_sum_df_thresh.shape)
power_sum_df_thresh.head(50)

In [ ]:
import math
def facet_plot(image_df, time_stamp_list_flatness):
    plt.figure(figsize = (20,20))
    plot_row_num = math.ceil(math.sqrt(len(time_stamp_list_flatness)))
    plot_col_num = math.ceil(len(time_stamp_list_flatness) / plot_row_num)
    for idx, time_stamp in enumerate(time_stamp_list_flatness):
        plt.subplot(plot_row_num, plot_col_num, idx+1)
        plt.pcolormesh((times*1000) + (time_stamp), freqs_spec / 1000, 10 * np.log10(image_df[time_stamp]))
    plt.show()

In [ ]:
facet_plot(spec_slices, time_stamp_list_flatness)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=30, whiten=True)
X_transformed = pca.fit_transform(np.log(annot_slice_df))

In [ ]:
X_transformed.shape

In [ ]:
pca.components_.shape

In [ ]:
plt.plot(pca.singular_values_)
plt.title("Singular values")

In [ ]:
pca.components_[:,:].shape

In [ ]:
pca.components_[0]

In [ ]:
# plot first ten components
plt.figure(figsize = (10,5))
u = pca.components_[:,:].reshape(30,-1,len(times))
for i in range(10):
    plt.subplot(2,5,i+1)
    # we are rescaling between 0 and 1 before plotting
    plt.imshow(u[i], cmap = 'viridis')
    plt.title('Mode '+str(i+1))

In [ ]:
plt.scatter(X_transformed[:,0], X_transformed[:,1], s=40,  marker='o', alpha=.8)
#plt.xlim(-100000,100000)
#plt.ylim(-100000,100000)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
# The code below visualizes the data projected in 2D together with the original spectrograms

from matplotlib import offsetbox
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
# Scale and visualize the embedding vectors
def plot_embedding(X, X_embedded, title=None):
    x_min, x_max = np.min(X_embedded, 0), np.max(X_embedded, 0)
    X_embedded = (X_embedded - x_min) / (x_max - x_min)

    plt.figure(figsize = (10,10))
    ax = plt.subplot(111)
    for i in range(X_embedded.shape[0]):
        plt.text(X_embedded[i, 0], X_embedded[i, 1], 'o',
                 color=plt.cm.viridis(y[i] / 0.01),
                 #color=colors[i],
                 #color=int(y[i]),
                 fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X_embedded.shape[0]):
            dist = np.sum((X_embedded[i,:] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X_embedded[i,:]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(np.log(X[i,:].reshape(spec_dim)),zoom = 0.3, cmap='plasma'),
                X_embedded[i,:])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
plot_embedding(np.array(annot_slice_df), X_transformed[:,:2])

In [ ]:
# center and scale the data
scaler = StandardScaler()
slices_scaled = scaler.fit_transform(spec_slices_df)

In [ ]:
k_range = range(2,20)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(slices_scaled)
    scores.append(silhouette_score(slices_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
km2 = KMeans(n_clusters=3,random_state=1234)
km2.fit(slices_scaled)
score = silhouette_score(slices_scaled, km_ss.labels_)
#summary_ave['kmeans_2_scaled'] = [ "cluster_" + str(label) for label in km2.labels_ ]
#summary_ave.groupby('kmeans_2_scaled').mean()
print(score)

Create df of annotated USVs from RavenLite

In [ ]:
annot_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/USV/USV_annot.csv"

In [ ]:
data = pd.read_csv(annot_path)
annot_data = pd.DataFrame(data = data)
annot_data.head()

In [ ]:
annot_data.groupby(['Animal', 'Session', 'Annotation']).describe()

Determine closest time stamp of each annotation and add as column to df

In [ ]:
annot_data['Begin Time (s)_1000'] = annot_data['Begin Time (s)']*1000

In [ ]:
annot_data.head()

In [ ]:
annot_time_stamps = []
values = annot_data['Begin Time (s)_1000'].values
for value in values:
    time_stamp_num = int(value / 22.5)
    time_stamp_index = time_stamp_num*22.5
    annot_time_stamps.append(time_stamp_index)

annot_data['time_stamp'] = annot_time_stamps

In [ ]:
annot_data.head()

In [ ]:
annot_535 = annot_data[(annot_data['Animal'] == 535)]
annot_535.reset_index(inplace = True)
annot_535.head()

In [ ]:
annot_535_low_multi = annot_535[annot_535['Annotation'].str.contains('low multi', regex=False)]
print(annot_535_low_multi.shape)
annot_535_low_multi.head()

In [ ]:
annot_535_low_multi_slices = spec_slices_df.loc[annot_535_low_multi['time_stamp']]
print(annot_535_low_multi_slices.shape)
annot_535_low_multi_slices.head()

In [ ]:
annot_features_df_535_low_multi_slices = find_features(annot_535_low_multi_slices)
print(annot_features_df_535_low_multi_slices.shape)
annot_features_df_535_low_multi_slices

In [ ]:
print(annot_features_df_535_low_multi_slices['spec_flat'].max())
print(annot_features_df_535_low_multi_slices['spec_flat'].min())
print(annot_features_df_535_low_multi_slices['spec_flat'].mean())
print(annot_features_df_535_low_multi_slices['power_sum'].max())
print(annot_features_df_535_low_multi_slices['power_sum'].min())
print(annot_features_df_535_low_multi_slices['power_sum'].mean())

In [ ]:
time_stamp_list = annot_slice_df.index.values

In [ ]:
multi_plot(spec_slices, time_stamp_list)

In [ ]:
plt.hist(annot_features_df_535_low_multi_slices['power_sum'].values)
plt.show()